## Huffman Encoding

In [1]:
import numpy as np
import matplotlib.pyplot as plt


### Single Counts processing
This is required for Huffman Encoding to count the apperances of each of the utf-8 characters

In [2]:
def get_single_counts(data):
    #returns a dictionary of single character counts
    #inputs: data: string
    #outputs: dictionary of single character counts
    single_counts = {}
    for char in data:
        #encode char in utf-8 
        if char not in single_counts:
            single_counts[char] = 0
        single_counts[char] += 1
    print(f"single_counts: {single_counts}")
    return single_counts
        

def get_n_counts(data, n, debug = False):
    n_counts = {}
    for i in range(len(data)-n+1):
        # if debug:
        #     print(f"i: {i}, data[i:i+n]: {data[i:i+n]}")
        n_gram = data[i:i+n]
        if n_gram not in n_counts:
            n_counts[n_gram] = 0
        n_counts[n_gram] += 1
    
    if debug:
        print(f"{n}-gram counts: {n_counts}")
    return n_counts

def get_disjoint_n_counts(data, n, debug=False):
    n_counts = {}
    #get n-gram counts for disjoint blocks of the data size n and final step we just get the remainder gram
    for i in range(0, len(data)-n+1, n):
        n_gram = data[i:i+n]
        if n_gram not in n_counts:
            n_counts[n_gram] = 0
        n_counts[n_gram] += 1
    #final step
    if len(data)%n != 0:
        n_gram = data[-(len(data)%n):]
        if n_gram not in n_counts:
            n_counts[n_gram] = 0
        n_counts[n_gram] += 1
    return n_counts


### Create Huffman Class

In [3]:
#create tree class:
from requests import get


class Tree:
    def __init__(self, left, right, freq, value= None):
        self.left = left
        self.right = right
        self.freq = freq
        self.value = value

    def __lt__(self, other):
        return self.freq < other.freq

    def __repr__(self):
        return f"Tree({self.freq}, {self.value}, {self.left}, {self.right})"
    

class Huffman:
    def __init__(self, counts =1, disjoint = True):
        #Huffman Model, including encoder and decoder, codebook difference, and compression ratio
        #data: string to be encoded
        #counts: number of characters to encode at a time, default 1
        #disjoint: boolean to encode disjoint blocks of the data, default False
        self.huffman_tree = None
        self.codebook = {}
        self.counts = counts
        self.counts_dict = {}
        self.disjoint = disjoint
        
    def run(self, data, file_name=None, debug = False):
        if file_name is not None:
            print(f"-------------------\nRunning Huffman Model on input file: {file_name} of length: {len(data)}\n-------------------")
        
        else: print(f"---------------------\nRunning Huffman model on data of length {len(data)}\n---------------------")
        print(f"counts: {self.counts}, disjoint: {self.disjoint}")
        if debug:
            print(f"counts: {self.counts}, disjoint: {self.disjoint}")
        if len(data) < self.counts:
            print(f"Data length must be greater than {self.counts}, as the model uses {self.counts}-grams")
            return None
        code = self.encode(data, debug)
        output = self.decode(code, debug)  
        print(f"Input-Ouput Verification: {output == data}") 
        print(f"Compression ratio: {self.get_compression_ratio(data, code)}")
        try: 
            if debug.lower() == "verbose":
                print(f"code: {code}")
                print(f"Input: {data}\nOutput: {output}")

        except:
            if debug:
                print(f"Input: {data}\nOutput: {output}")
      
        
    def encode(self, data, debug = False):
        #encode the data using the huffman tree
        #inputs: data: string of the data to be encoded
        #        single_counts: dictionary of single character counts
        #outputs: encoded data

        self.get_counts(data,debug) #obtains self.counts_dict
        if debug:
            print(f"{self.counts}-gram counts: {self.counts_dict}")
        self.generate_huffman_tree(debug=debug) #obtains self.huffman_tree
        if debug:
            print(f"huffman_tree: {self.huffman_tree}")
        self.generate_codebook() #obtains self.codebook
        if debug:
            print(f"encoder_codebook: {self.codebook}")
        code = self.get_code(data) #obtains code
        return code

    def get_code(self, data):
        #returns the code of the data
        #inputs: data: string of the data to be encoded
        #        huffman_tree: the huffman tree to encode the data
        #outputs: code: encoded data
        if self.counts == 1 or not self.disjoint:
            code = ""
            #encode n-gramwise
            for i in range(len(data)-self.counts +1):
                n_gram = data[i:i+self.counts]
                code += self.codebook[n_gram]
            
            return code
        else: #n-gramwise disjoint
            code = ""
            #encode with n-gramwise jumps
            for i in range(0, len(data), self.counts):
                n_gram = data[i:i+self.counts]
                code += self.codebook[n_gram]
            #final step
            return code
    
        
    def generate_codebook(self):
        #generate codebook from self.huffman_tree   

        def create_encoding(tree, code):
            if tree.value:
                self.codebook[tree.value] = code
            else:
                create_encoding(tree.left, code + '0')
                create_encoding(tree.right, code + '1')
        create_encoding(self.huffman_tree, '')
        
    
    def generate_huffman_tree(self, debug = False):
        if debug:
            print(f"counts_dict: {self.counts_dict}")
        trees = [Tree(None, None, self.counts_dict[key], key) for key in self.counts_dict]
        #2. sort the list based on frequency
        trees.sort()

        #3. while there is more than one tree in the list:
        while len(trees) > 1:
            #3.1 take the first two trees from the list and create a new tree with them as children. 
            #The new tree's frequency is the sum of the two trees' frequencies.
            #The two frequencies guaranteed to be the lowest two
            left = trees.pop(0)
            right = trees.pop(0)
            new_tree = Tree(left, right, left.freq + right.freq)
            #3.2 add the new tree to the list
            trees.append(new_tree)
            #3.3 re-sort the list for the priority queue to have the lowest frequency at the top
            trees.sort()
            
        #4. the tree left in the list is the huffman tree
        self.huffman_tree = trees[0]
        
    def decode(self, code, debug = False):
        #decode the code using the huffman tree
        #inputs: code: string of the code to be decoded
        #        huffman_tree: the huffman tree to decode the code
        #outputs: decoded data
        if self.counts == 1 or not self.disjoint:
            decoded_data = ""
            current_code = ""
            inverse_codebook = {v: k for k, v in self.codebook.items()}
            n_gram_size = self.counts  # Adjust for n-grams
            
            for bit in code:
                current_code += bit
                if current_code in inverse_codebook.keys():
                    if n_gram_size == 1:
                        decoded_data += inverse_codebook[current_code]
                        current_code = ""
                    else:
                        decoded_data  = decoded_data[:-n_gram_size+1] + inverse_codebook[current_code]
                        current_code = ""
            return decoded_data
        else: #disjoint
            decoded_data = ""
            current_code = ""
            inverse_codebook = {v: k for k, v in self.codebook.items()}
            n_gram_size = self.counts  # Adjust for n-grams
            for bit in code:
                current_code += bit
                
                if current_code in inverse_codebook.keys():
                    if debug:
                        print(f"current_code: {current_code}")
                    decoded_data += inverse_codebook[current_code]
                    current_code = ""
                    if debug:
                        print(f"decoded_data: {decoded_data}")
            return decoded_data
                    
        
    
    def get_counts(self, data, debug = False):
        #returns a dictionary of character counts based on self.counts
        #inputs: data: string
        #outputs: dictionary of self.counts # character counts
        if self.counts ==1 or not self.disjoint:
            if debug:
                print(f"get n counts")
            self.counts_dict = get_n_counts(data, self.counts, debug)
        else:
            if debug:
                print(f"get disjoint n counts")
            self.counts_dict = get_disjoint_n_counts(data, self.counts, debug)
        
        
    def get_compression_ratio(self, data, code):
        num_bits_data_utf8 = len(data.encode('utf-8'))*8
        num_bits_dictionary = sum([len(self.codebook[key]) + len(key.encode('utf-8'))*8 for key in self.codebook]) 
        #value is in binary, key is in utf-8, we add key + value bits for each key.
        num_bits_code = len(code) #code is already in bits.
        print(f"num_bits_data_utf8: {num_bits_data_utf8}, num_bits_code: {num_bits_code} num_bits_dictionary: {num_bits_dictionary}")
        compression_ratio = num_bits_data_utf8/ (num_bits_code+ num_bits_dictionary) 
        return compression_ratio
    
    def get_codebook_difference(self, encoder_codebook, decoder_codebook):
        #returns the difference between the codebooks of the encoder and decoder
        #returns a dictionary of the differences   
        difference = {}
        for key in encoder_codebook:
            if key not in decoder_codebook.values():
                difference[key] = encoder_codebook[key]
        return difference
    def get_data(self):
        return self.data
    def run_onfile(self, filename, debug = False):
        #runs the Huffman model on the input file
        #returns the compression ratio
        #Inputs: filename: string of the file to be encoded 
        #        debug [True, False, "Verbose"]: boolean to print debug statements
        #Prints: Input-Ouput Verification: True if the input and output are the same, False otherwise
        #        Compression ratio: the compression ratio of the input and output
        #        code: the encoded data
        #        encoder_codebook: the codebook of the encoder
        #        decoder_codebook: the codebook of the decoder
        with open(filename, "r") as f:
            data = f.read()
        self.run(data, filename, debug)

### Trade off for larger n-grams
Although it might sound good since we can output all in one codebook, our worst case if data is a sequence of unique n-nary elements, our codebook needs to store the each of the elements size of n, which means it's uncompressed.
### Observation Disjoint $\geq$ sliding window:
Implementation issue of sliding window is just inefficient since we have ```len(data)-n+1``` codewords no matter what instead of ```roof((len(data)/n))```



In [4]:
#test on 1,2,3 counts same as LZW
# Single counts:
import dis


huffman = Huffman(counts = 1)
huffman.run("".join([chr(i) for i in range(256)]))
# huffman.run("BABAABAAA")
# 2-gram counts:
huffman = Huffman(counts = 2, disjoint = True)
# huffman.run("AAB", debug = "verbose")   

huffman.run("TOBEORNOTTOBEORTOBEORNOT")
# huffman.run("BABAABAAA")
# # # 3-gram counts:
huffman = Huffman(counts = 3, disjoint =True)
huffman.run("TOBEORNOTTOBEORTOBEORNOT")
# huffman.run("BABAABAAA")
huffman = Huffman(counts = 8, disjoint =True)
huffman.run("TOBEORNOTTOBEORTOBEORNOT")
# huffman.run("BABAABAAA")


---------------------
Running Huffman model on data of length 256
---------------------
counts: 1, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 3072, num_bits_code: 2048 num_bits_dictionary: 5120
Compression ratio: 0.42857142857142855
---------------------
Running Huffman model on data of length 24
---------------------
counts: 2, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 192, num_bits_code: 40 num_bits_dictionary: 194
Compression ratio: 0.8205128205128205
---------------------
Running Huffman model on data of length 24
---------------------
counts: 3, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 192, num_bits_code: 13 num_bits_dictionary: 77
Compression ratio: 2.1333333333333333
---------------------
Running Huffman model on data of length 24
---------------------
counts: 8, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 192, num_bits_code: 5 num_bits_dictionary: 197
Compression ratio: 0.9504950495049

In [5]:
n = 10000
data = ''.join(np.random.choice(['A','B'], size=n))



In [6]:
Huffman(counts= 1, disjoint = True).run(data)
Huffman(counts = 2, disjoint = True).run(data)
Huffman(counts = 5, disjoint = True).run(data)
Huffman(counts = 10, disjoint = True).run(data)
Huffman(counts = 100, disjoint = True).run(data)



---------------------
Running Huffman model on data of length 10000
---------------------
counts: 1, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 80000, num_bits_code: 10000 num_bits_dictionary: 18
Compression ratio: 7.98562587342783
---------------------
Running Huffman model on data of length 10000
---------------------
counts: 2, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 80000, num_bits_code: 10000 num_bits_dictionary: 72
Compression ratio: 7.942811755361398
---------------------
Running Huffman model on data of length 10000
---------------------
counts: 5, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 80000, num_bits_code: 10000 num_bits_dictionary: 1440
Compression ratio: 6.993006993006993
---------------------
Running Huffman model on data of length 10000
---------------------
counts: 10, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 80000, num_bits_code: 9202 num_bits_dictionary: 58184
Compressi

## Run Huffman on filtered datasets

In [7]:
#run LZW on all files in filtered_datasets/books and filtered_datasets/music
import os
for n_gram in [1,2,4,8]:
    
    huffman = Huffman(counts = n_gram, disjoint = True)
    for root, dirs, files in os.walk("filtered_datasets/books"):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                try:
                    huffman.run_onfile(file_path)
                except:
                    print(f"File: {file_path} failed on n_gram size: {n_gram}")
            
    for root, dirs, files in os.walk("filtered_datasets/music"):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)

                try:
                    huffman.run_onfile(file_path)
                except:
                    print(f"File: {file_path} failed on n_gram size: {n_gram}")

-------------------
Running Huffman Model on input file: filtered_datasets/books/principia_mathematica.txt of length: 804324
-------------------
counts: 1, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 6434592, num_bits_code: 3745936 num_bits_dictionary: 1469
Compression ratio: 1.7170794189579188
-------------------
Running Huffman Model on input file: filtered_datasets/books/Bible.txt of length: 4332587
-------------------
counts: 1, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 34660696, num_bits_code: 19827130 num_bits_dictionary: 1521
Compression ratio: 1.7480107950863626
-------------------
Running Huffman Model on input file: filtered_datasets/books/beowulf.txt of length: 273007
-------------------
counts: 1, disjoint: True
Input-Ouput Verification: True
num_bits_data_utf8: 2184056, num_bits_code: 1226832 num_bits_dictionary: 1520
Compression ratio: 1.7780375657791903
-------------------
Running Huffman Model on input file: filtered_dataset